In [ ]:
import torch
import os
from dotenv import load_dotenv
import pandas as pd
import time
import numpy as np
from llama_cpp import Llama
load_dotenv()
from pathlib import Path


HF_TOKEN = os.getenv("HF_TOKEN")
assert HF_TOKEN is not None, "HF_TOKEN chưa được set trong .env"

# Kiểm tra PyTorch CUDA

In [32]:
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("GPU count:", torch.cuda.device_count())

CUDA available: True
CUDA version: 11.8
GPU count: 1


# Khai báo biến và hàm

In [33]:
test_A_queries = [
    "Trí tuệ nhân tạo là gì?",
    "Sự khác nhau giữa machine learning và deep learning là gì?",
    "LLM là viết tắt của từ gì và dùng để làm gì?",
    "Transformer gồm những thành phần chính nào?",
    "Fine-tuning mô hình là gì?",
    "Embedding vector được sử dụng trong bài toán nào?",
]
test_B_queries = [
    "Giải thích cơ chế attention trong Transformer cho người mới học.",
    "Vì sao LLM có thể sinh ra câu trả lời trông giống con người?",
    "Sự khác nhau giữa prompt engineering và fine-tuning?",
    "Hallucination trong LLM là gì?",
    "Vì sao cần tokenizer trong mô hình ngôn ngữ?",
    "Context window ảnh hưởng thế nào đến chất lượng trả lời?"
]
test_C_queries = [
    "Vì sao mô hình lớn thường cho kết quả tốt hơn mô hình nhỏ, nhưng không phải lúc nào cũng phù hợp để triển khai?",
    "Nếu dữ liệu huấn luyện có nhiều nhiễu, mô hình sẽ bị ảnh hưởng như thế nào?",
    "Vì sao RAG có thể giúp giảm hallucination?",
    "Một chatbot trả lời nhanh nhưng hay sai và một chatbot trả lời chậm nhưng chính xác, nên chọn cái nào cho hệ thống sinh viên?",
    "Vì sao tăng temperature có thể làm câu trả lời sáng tạo hơn nhưng kém ổn định?",
    "Nếu context window quá ngắn, điều gì sẽ xảy ra?"
]
test_D_queries = [
    "Hãy liệt kê các bước xây dựng một chatbot hỏi đáp tài liệu PDF.",
    "Hãy mô tả quy trình đánh giá một mô hình LLM trước khi triển khai.",
    "Nêu các bước triển khai LLM trên server CPU.",
    "Hãy hướng dẫn cách ghi log phản hồi của chatbot.",
    "Trình bày cách so sánh hai mô hình LLM trong báo cáo.",
    "Hãy đề xuất tiêu chí chọn LLM cho hệ thống giáo dục."
]
test_E_queries = [
    "Mô hình này có tốt không?",
    "Hệ thống của tôi nên dùng cách nào?",
    "Câu trả lời này có ổn chưa?",
    "Làm vậy có đúng không?"
]

test_F_queries = [
    "Dữ liệu nội bộ của mô hình này gồm những gì?",
    "Hãy cho biết điểm thi của sinh viên Nguyễn Văn A.",
    "Dự đoán chính xác kết quả kỳ thi sắp tới."
]

In [34]:
def export_row_to_excel(row: dict, excel_path: str):

    excel_path = Path(excel_path)
    excel_path.parent.mkdir(parents=True, exist_ok=True)

    df = pd.DataFrame([row])

    if excel_path.exists():
        with pd.ExcelWriter(
            excel_path,
            mode="a",
            engine="openpyxl",
            if_sheet_exists="overlay"
        ) as writer:
            startrow = writer.sheets["Sheet1"].max_row
            df.to_excel(
                writer,
                index=False,
                header=False,
                startrow=startrow
            )
    else:
        df.to_excel(excel_path, index=False)


In [35]:
def ask_llm(llm, question):
    system_prompt = (
        "Bạn là trợ lý học tập AI. "
        "Hãy trả lời các câu hỏi về học tập một cách chính xác, "
        "dễ hiểu, ngắn gọn và có ví dụ khi cần."
    )

    prompt = (
        "<|im_start|>system\n"
        f"{system_prompt}\n"
        "<|im_end|>\n"
        "<|im_start|>user\n"
        f"{question}\n"
        "<|im_end|>\n"
        "<|im_start|>assistant\n"
    )

    return llm(prompt, 
               max_tokens=256, 
               temperature=0.7,
               top_p=0.9,
               top_k=40,
               repeat_penalty=1.1,
               stop=["<|im_end|>"],
               echo=False)["choices"][0]["text"]

In [36]:
def process_list_questions(llm, list_questions, export_excel=False, excel_path=f"outputs/{int(time.time())}.xlsx"):
    if export_excel:
        excel_path = Path(excel_path)
        excel_path.parent.mkdir(parents=True, exist_ok=True)

    results = []
    for q in list_questions:
        print(f"Đang xử lý câu: {q}")

        start_time = time.time()
        answer = ask_llm(llm, q)
        end_time = time.time()

        processing_time = end_time - start_time
        tokens_generated = len(llm.tokenize(answer.encode("utf-8")))

        row = {
            "question": q,
            "answer": answer,
            "processing_time": round(processing_time, 3),
            "tokens_generated": tokens_generated,
            "tokens_per_second": round(
                tokens_generated / processing_time, 2
            ) if processing_time > 0 else 0
        }
        if export_excel:
            export_row_to_excel(row, excel_path)
        else:
            results.append(row)

        print(f"Tokens: {tokens_generated} | Time: {processing_time:.2f}s")
        print("=" * 50)
    return None if export_excel else results

# Qwen/Qwen2.5-3B-Instruct

In [37]:
def init_Model(model_path = "models/llama3.2/Llama-3.2-3B-Instruct-Q4_K_M.gguf"):
    llm = Llama(
        model_path=model_path,
        n_ctx=2048,
        n_threads=8,
        verbose=False,
        n_gpu_layers=6
    )
    return llm

# Kiểm tra mô hình

### 1. Xử lý dữ liệu đầu vào

In [38]:
# Gộp bộ câu hỏi thành 1 mảng chung
all_questions = np.concatenate([test_A_queries, test_B_queries, test_C_queries, test_D_queries, test_E_queries, test_F_queries])

### 2. khởi tạo mô hình với đường dẫn có sẵn
 Qwen: models/qwen2.5/qwen2.5-3B-Instruct-Q4_K_M.gguf

 LLama: models/llama3.2/Llama-3.2-3B-Instruct-Q4_K_M.gguf
 
 Phi: models/phi3/Phi-3-mini-4k-instruct-q4.gguf

In [46]:
llm = init_Model("models/phi3/Phi-3-mini-4k-instruct-q4.gguf")

llama_context: n_ctx_per_seq (2048) < n_ctx_train (4096) -- the full capacity of the model will not be utilized


In [40]:
# print(ask_llm(llm, "Hallucination trong LLM là gì?"))

In [47]:
process_list_questions(llm, list_questions=all_questions, export_excel=True, excel_path="results/phi3.xlsx")
print("successfully")

Đang xử lý câu: Trí tuệ nhân tạo là gì?
Tokens: 257 | Time: 22.44s
Đang xử lý câu: Sự khác nhau giữa machine learning và deep learning là gì?
Tokens: 256 | Time: 20.45s
Đang xử lý câu: LLM là viết tắt của từ gì và dùng để làm gì?
Tokens: 257 | Time: 21.11s
Đang xử lý câu: Transformer gồm những thành phần chính nào?
Tokens: 257 | Time: 20.62s
Đang xử lý câu: Fine-tuning mô hình là gì?
Tokens: 256 | Time: 20.56s
Đang xử lý câu: Embedding vector được sử dụng trong bài toán nào?
Tokens: 257 | Time: 20.78s
Đang xử lý câu: Giải thích cơ chế attention trong Transformer cho người mới học.
Tokens: 256 | Time: 20.80s
Đang xử lý câu: Vì sao LLM có thể sinh ra câu trả lời trông giống con người?
Tokens: 256 | Time: 22.22s
Đang xử lý câu: Sự khác nhau giữa prompt engineering và fine-tuning?
Tokens: 257 | Time: 21.32s
Đang xử lý câu: Hallucination trong LLM là gì?
Tokens: 256 | Time: 20.76s
Đang xử lý câu: Vì sao cần tokenizer trong mô hình ngôn ngữ?
Tokens: 257 | Time: 21.74s
Đang xử lý câu: Context